# Predicting Parkinsons and Atypical Parkinsons

### Import Libraries and set random seed

In [67]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Ignore annoying warnings about future deprecations 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()

### Set Random Number Seed
np.random.seed(112)

### Load the Data

In [50]:
data = pd.read_excel("Budathon_2018.xlsx")
data.head()

,Age,Sex,UPDRS,aSN_FA,Caudate_FA,CC_FA,GP_FA,LobuleVI_FA,LobuleV_FA,MCP_FA,...,LobuleVI_FW,LobuleV_FW,MCP_FW,pSN_FW,Putamen_FW,SCP_FW,STN_FW,Thalamus_FW,Vermis_FW,GroupID
0,68,1,0.0,0.645260,0.237540,0.658996,0.390937,0.199762,0.248622,0.583646,...,0.188899,0.253751,0.098390,0.173284,0.159989,0.415654,0.124107,0.160673,0.204663,0
1,53,1,0.0,0.654966,0.222827,0.607287,0.383109,0.225891,0.275686,0.635246,...,0.133954,0.140164,0.086658,0.195432,0.169670,0.308992,0.142579,0.149033,0.146168,0
2,67,0,1.0,0.674251,0.236799,0.743437,0.324580,0.224945,0.237046,0.636315,...,0.228307,0.299650,0.078886,0.137439,0.185490,0.354840,0.096919,0.140484,0.202398,0
3,46,1,4.7,0.664740,0.221283,0.702845,0.346713,0.224896,0.258485,0.632627,...,0.195933,0.161162,0.081963,0.172358,0.122145,0.380896,0.085467,0.080924,0.167342,0
4,45,0,2.0,0.656800,0.204532,0.692499,0.322482,0.250622,0.225412,0.575726,...,0.131577,0.170267,0.085887,0.244718,0.141260,0.427207,0.117136,0.156742,0.169126,0


## Controls vs Any Parkinsonism

### Group the Parkinsons subjects into a single common group

In [71]:
df = data.copy()
df.loc[df['GroupID'] != 0, 'GroupID'] = 1 

### Split into train and test sets

In [90]:
x_cols = [col for col in df.columns if col != 'GroupID']
X = pd.DataFrame(df, columns= x_cols)
Y = df['GroupID']

### Build and compile the keras model

In [110]:
input_len = len(X.columns)

model = Sequential()
model.add(Dense(input_len, input_dim=input_len, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',auc_roc])

C:\Users\jtbricker\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


### Deine some custom functions to measure AUC

In [107]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

In [ ]:
### Run and evaluate the model

In [111]:
model.fit(X, Y, epochs=150, batch_size=10)

scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

C:\Users\jtbricker\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Epoch 1/150
746/746 [==============================] - 1s 1ms/step - loss: 0.6841 - acc: 0.8043 - auc: 0.7304
Epoch 2/150
746/746 [==============================] - 0s 302us/step - loss: 0.1951 - acc: 0.9383 - auc: 0.8844
Epoch 3/150
746/746 [==============================] - 0s 306us/step - loss: 0.1768 - acc: 0.9504 - auc: 0.9221
Epoch 4/150
746/746 [==============================] - 0s 272us/step - loss: 0.1699 - acc: 0.9544 - auc: 0.9375
Epoch 5/150
746/746 [==============================] - 0s 258us/step - loss: 0.1726 - acc: 0.9450 - auc: 0.9481
Epoch 6/150
746/746 [==============================] - 0s 298us/step - loss: 0.1653 - acc: 0.9531 - auc: 0.9529
Epoch 7/150
746/746 [==============================] - 0s 292us/step - loss: 0.1645 - acc: 0.9477 - auc: 0.9572
Epoch 8/150
746/746 [==============================] - 0s 265us/step - loss: 0.1597 - acc: 0.9464 - auc: 0.9605
Epoch 9/150
746/746 [==============================] - 0s 268us/step - loss: 0.1666 - acc: 0.9491 - auc: 0